First downlaod the module pytrends.

```
# pip install pytrends

```

Cite: https://github.com/qztseng/google-trends-daily/blob/master/gtrend.py



In [1]:
!pip install pytrends

In [2]:
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
from datetime import datetime, timedelta, date, time
import pandas as pd
import time

In [3]:
def _fetch_data(trendreq, kw_list, timeframe='today 3-m', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retry the same(in case of a ResponseError)."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_over_time()


In [4]:
def get_daily_trend(trendreq, keyword:str, start:str, end:str, cat=0, 
                    geo='', gprop='', delta=269, overlap=100, sleep=0, 
                    tz=0, verbose=False) ->pd.DataFrame:

    """Stich and scale consecutive daily trends data between start and end date.
    This function will first download piece-wise google trends data and then 
    scale each piece using the overlapped period. 
        Parameters
        ----------
        trendreq : TrendReq(hl='EN', tz=360)
            a pytrends TrendReq object, here hl selects language and tz is timezone offset
        keyword: str
            Enter your keyword to find trends for
        start: str
            starting date in string format:YYYY-MM-DD (e.g.2017-02-19)
        end: str
            ending date in string format:YYYY-MM-DD (e.g.2017-02-19)
        cat, geo, gprop, sleep: 
            Important is geo which means geographical area. Default is worldwide
            gprop means where to search like images, web, etc. Default is web
        delta: int
            The length(days) of each timeframe fragment for fetching google trends data, 
            need to be <269 in order to obtain daily data.
        overlap: int
            The length(days) of the overlap period used for scaling/normalization
        tz: int
            The timezone shift in minute relative to the UTC+0 (google trends default).
            For example, correcting for UTC+8 is 480, and UTC-6 is -360 
    """
    
    start_d = datetime.strptime(start, '%Y-%m-%d')
    init_end_d = end_d = datetime.strptime(end, '%Y-%m-%d')
    init_end_d.replace(hour=23, minute=59, second=59)    
    delta = timedelta(days=delta)
    overlap = timedelta(days=overlap)

    itr_d = end_d - delta
    overlap_start = None

    df = pd.DataFrame()
    ol = pd.DataFrame()
    
    while end_d > start_d:
        tf = itr_d.strftime('%Y-%m-%d')+' '+end_d.strftime('%Y-%m-%d')
        if verbose: print('Fetching \''+keyword+'\' for period:'+tf)
        temp = _fetch_data(trendreq, [keyword], timeframe=tf, cat=cat, geo=geo, gprop=gprop)
        temp.drop(columns=['isPartial'], inplace=True)
        temp.columns.values[0] = tf
        ol_temp = temp.copy()
        ol_temp.iloc[:,:] = None
        if overlap_start is not None:  # not first iteration
            if verbose: print('Normalize by overlapping period:'+overlap_start.strftime('%Y-%m-%d'), end_d.strftime('%Y-%m-%d'))
            #normalize using the maximum value of the overlapped period
            y1 = temp.loc[overlap_start:end_d].iloc[:,0].values.max()
            y2 = df.loc[overlap_start:end_d].iloc[:,-1].values.max()
            coef = y2/y1
            temp = temp * coef
            ol_temp.loc[overlap_start:end_d, :] = 1 

        df = pd.concat([df,temp], axis=1)
        ol = pd.concat([ol, ol_temp], axis=1)
        # shift the timeframe for next iteration
        overlap_start = itr_d
        end_d -= (delta-overlap)
        itr_d -= (delta-overlap)
        # in case of short query interval getting banned by server
        time.sleep(sleep)
    
    df.sort_index(inplace=True)
    ol.sort_index(inplace=True)
    #The daily trend data is missing the most recent 3-days data, need to complete with hourly data
    if df.index.max() < init_end_d : 
        tf = 'now 7-d'
        hourly = _fetch_data(trendreq, [keyword], timeframe=tf, cat=cat, geo=geo, gprop=gprop)
        hourly.drop(columns=['isPartial'], inplace=True)
        
        #convert hourly data to daily data
        daily = hourly.groupby(hourly.index.date).sum()
        
        #check whether the first day data is complete (i.e. has 24 hours)
        daily['hours'] = hourly.groupby(hourly.index.date).count()
        if daily.iloc[0].loc['hours'] != 24: daily.drop(daily.index[0], inplace=True)
        daily.drop(columns='hours', inplace=True)
        
        daily.set_index(pd.DatetimeIndex(daily.index), inplace=True)
        daily.columns = [tf]
        
        ol_temp = daily.copy()
        ol_temp.iloc[:,:] = None
        # find the overlapping date
        intersect = df.index.intersection(daily.index)
        if verbose: print('Normalize by overlapping period:'+(intersect.min().strftime('%Y-%m-%d'))+' '+(intersect.max().strftime('%Y-%m-%d')))
        # scaling use the overlapped today-4 to today-7 data
        coef = df.loc[intersect].iloc[:,0].max() / daily.loc[intersect].iloc[:,0].max()
        daily = (daily*coef).round(decimals=0)
        ol_temp.loc[intersect,:] = 1
        
        df = pd.concat([daily, df], axis=1)
        ol = pd.concat([ol_temp, ol], axis=1)

    # taking averages for overlapped period
    df = df.mean(axis=1)
    ol = ol.max(axis=1)
    # merge the two dataframe (trend data and overlap flag)
    df = pd.concat([df,ol], axis=1)
    df.columns = [keyword, 'overlap']
    # Correct the timezone difference
    df.index = df.index + timedelta(minutes=tz)
    df = df[start_d:init_end_d]
    # re-normalized to the overall maximum value to have max =100
    df[keyword] = (100*df[keyword]/df[keyword].max()).round(decimals=0)

    df.drop('overlap', axis='columns', inplace=True)
    
    return df

In [5]:
df = get_daily_trend(TrendReq(hl='EN', tz=360),'bitcoin', '2013-10-01', '2021-3-31', cat = 0, geo='', gprop='')

In [6]:
df

,bitcoin
date,
2013-10-01,1.0
2013-10-02,2.0
2013-10-03,3.0
2013-10-04,2.0
2013-10-05,1.0
...,...
2021-03-27,18.0
2021-03-28,18.0
2021-03-29,19.0


In [14]:
df.to_csv('bitcoin.csv')

In [15]:
df_1 = get_daily_trend(TrendReq(hl='EN', tz=360),'cardano', '2018-05-31', '2021-4-12', cat = 0, geo='', gprop='')

In [16]:
df_1

,cardano
date,
2018-05-31,5.0
2018-06-01,4.0
2018-06-02,3.0
2018-06-03,4.0
2018-06-04,4.0
...,...
2021-04-08,28.0
2021-04-09,25.0
2021-04-10,29.0


In [17]:
df_2 = get_daily_trend(TrendReq(hl='EN', tz=360),'ETH', '2015-08-9', '2021-3-31', cat = 0, geo='', gprop='')

In [18]:
df_2

,ETH
date,
2015-08-09,7.0
2015-08-10,8.0
2015-08-11,6.0
2015-08-12,7.0
2015-08-13,6.0
...,...
2021-03-27,42.0
2021-03-28,42.0
2021-03-29,48.0


In [29]:
df_3 = get_daily_trend(TrendReq(hl='EN', tz=360),'litecoin', '2018-05-31', '2021-4-12', cat = 0, geo='', gprop='')

In [30]:
df_3

,litecoin
date,
2018-05-31,15.0
2018-06-01,16.0
2018-06-02,15.0
2018-06-03,16.0
2018-06-04,16.0
...,...
2021-04-08,39.0
2021-04-09,30.0
2021-04-10,39.0


In [21]:
df_4 = get_daily_trend(TrendReq(hl='EN', tz=360),'XLM', '2018-05-31', '2021-4-12', cat = 0, geo='', gprop='')

In [22]:
df_4

,XLM
date,
2018-05-31,6.0
2018-06-01,5.0
2018-06-02,6.0
2018-06-03,5.0
2018-06-04,5.0
...,...
2021-04-08,22.0
2021-04-09,17.0
2021-04-10,35.0


In [23]:
df_5 = get_daily_trend(TrendReq(hl='EN', tz=360),'XMR', '2018-05-31', '2021-4-12', cat = 0, geo='', gprop='')

In [24]:
df_5

,XMR
date,
2018-05-31,33.0
2018-06-01,32.0
2018-06-02,55.0
2018-06-03,35.0
2018-06-04,29.0
...,...
2021-04-08,52.0
2021-04-09,50.0
2021-04-10,55.0


In [27]:
df_6 = get_daily_trend(TrendReq(hl='EN', tz=360),'ripple', '2018-05-31', '2021-4-12', cat = 0, geo='', gprop='')

In [28]:
df_6

,ripple
date,
2018-05-31,20.0
2018-06-01,21.0
2018-06-02,19.0
2018-06-03,21.0
2018-06-04,24.0
...,...
2021-04-08,36.0
2021-04-09,31.0
2021-04-10,58.0


In [31]:
df_1.to_csv('ADA.csv')
df_2.to_csv('ETH.csv')
df_3.to_csv('LTC.csv')
df_4.to_csv('XLM.csv')
df_5.to_csv('XMR.csv')
df_6.to_csv('XRP.csv')